In [1]:
%cd "/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt/"

# Third-party imports
import pyansys
import pandas as pd
import numpy as np
from pyOpt import Optimization
from pyOpt import ALPSO
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
%matplotlib widget
import itkwidgets

# Local imports
# from femodel import Femodel
from util_mapdl import Material
from util_mapdl.post_functions import fc_puck

/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt


In [2]:
class Femodel:
    
    def __init__(self, mapdl, mesh_factor=0.2):
        self.mapdl = mapdl
        self.__ansys_input_filename = '_ansys_input_file'
        self.mesh_density_factor = mesh_factor

    def cdread(self):
        """
        Read in the .cdb database file generated by the pre-processing 
        method.

        """
        
        self.mapdl.prep7()
        self.mapdl.cdread('all', self.__ansys_input_filename, 'cdb')        
        
    def clear(self):
        """Resets the MAPDL Session. """
        
        self.mapdl.finish()
        self.mapdl.clear('NOSTART')
        
    def pre_processing(self):

        self.mapdl.prep7() # Enter Preprocessing Routine
        
        # Space for basic parameters and settings
        self.mapdl.seltol(1e-4)
        self.mapdl.et('1','SHELL281')    
        self.mapdl.keyopt(1,8,1)

        # Material Parameters
        for key in self.materials:
            self.materials[key].assign_mp()

        # Abmaße des Balkens
        l = 1000 
        b = 10
        h = 30
    
        with self.mapdl.chain_commands:
            self.mapdl.et('1','SHELL281')
            self.mapdl.keyopt(1,8,1)
            
            # Modellgeometrie-Erstellung
            self.mapdl.k(1,0,0,h/2)
            self.mapdl.k(2,b,0,h/2)
            self.mapdl.k(3,b,l,h/2)
            self.mapdl.k(4,0,l,h/2)
            
            self.mapdl.k(5,0,0,-h/2)
            self.mapdl.k(6,b,0,-h/2)
            self.mapdl.k(7,b,l,-h/2)
            self.mapdl.k(8,0,l,-h/2)    
            
            self.mapdl.a(1,2,3,4)
            self.mapdl.a(5,6,7,8)    
            self.mapdl.a(6,7,3,2)
            
            # Mesh-Seeds
            self.mapdl.lsel('s','loc','x',b/2)
            self.mapdl.lesize('all','','',self.mesh_density_factor*b)
            
            self.mapdl.lsel('s','loc','y',l/2)
            self.mapdl.lesize('all','','',self.mesh_density_factor*l)
            
            self.mapdl.lsel('s','loc','z',0)
            self.mapdl.lesize('all','','',self.mesh_density_factor*h)
            
            # Vernetzung und Zuweisung Sections
            self.mapdl.mshkey(1)
            self.mapdl.mshape(0,'2d')
            self.mapdl.allsel('all')
            
            self.mapdl.sectype(100,'shell','','Dummy')
            self.mapdl.secdata(0.1,1,90.,3)
            self.mapdl.allsel('all')
            self.mapdl.amesh('all')
            
        self.mapdl.allsel('all')
        self.mapdl.cdwrite('all', self.__ansys_input_filename, 'cdb')
        self.mapdl.finish()
        self.mapdl.clear('nostart')
        
    def change_design_variables(self, divisions, topspar, bottomspar, web):

        # Abmaße des Balkens
        l = 1000 
        b = 10
        h = 30        
        
        # Überprüfen und sortieren der Trennstellen
        divisions = set(divisions)
        divisions = list(np.sort(list(divisions)))
        divisions.insert(0, 0)
        divisions.append(l)
        intervals = []
        for x in range(len(divisions)-1):
            intervals.append([divisions[x], divisions[x+1]])
    
            
        with self.mapdl.chain_commands:
            def secnum():
                secnum.counter+=1
                return secnum.counter
            secnum.counter = 0
    
            for i in intervals:       
                index = intervals.index(i)
                
                self.mapdl.sectype(secnum(),'shell','','Top_spar')
                self.mapdl.secdata(topspar[index],1,90.,3)
                self.mapdl.esel('s','cent','z',h/2)
                self.mapdl.esel('r','cent','y',i[0],i[1])
                self.mapdl.emodif('all','secnum',secnum.counter)
                
                self.mapdl.sectype(secnum(),'shell','','Bottom_spar')
                self.mapdl.secdata(bottomspar[index],1,90.,3)
                self.mapdl.esel('s','cent','z',-h/2)
                self.mapdl.esel('r','cent','y',i[0],i[1])
                self.mapdl.emodif('all','secnum',secnum.counter)
                
                self.mapdl.sectype(secnum(),'shell','','Shearweb')
                self.mapdl.secdata(web[index]/2,1,45.,3)                     #### achtung /2
                self.mapdl.secdata(web[index]/2,1,-45.,3)
                self.mapdl.esel('s','cent','x',b)
                self.mapdl.esel('r','cent','y',i[0],i[1])
                self.mapdl.emodif('all','secnum',secnum.counter)
                
            # Lastdefinition und Lösung
            # RBD
            self.mapdl.nsel('s','loc','x',0)
            self.mapdl.d('all','ux',0)
            self.mapdl.nsel('s','loc','y',0)
            self.mapdl.d('all','all',0)
            # Last
            self.mapdl.esel('s','cent','z',h/2)
            self.mapdl.sfe('all','','pres','0',-0.05/20)
            self.mapdl.esel('s','cent','z',-h/2)
            self.mapdl.sfe('all','','pres','0',-0.05/20)
            self.mapdl.allsel('all')
        
    def solve(self):
        """Solve the FE-Model. """
        
        self.mapdl.run('/SOLU')
        self.mapdl.antype('static')
        self.mapdl.outres('all','all')
        self.mapdl.solve()
        self.mapdl.finish()
        
    def post_processing(self, divisions):
        
        self.mapdl.post1() # Enter Post-processing Routine
        
        # Assign Failure Criteria Values
        for key in self.materials:
            self.materials[key].assign_fc()
        
        self.mapdl.fctyp('dele','all')
        self.mapdl.fctyp('add','pfib')
        self.mapdl.fctyp('add','pmat')        
        
        self.mapdl.allsel('all')
        
        # Abmaße des Balkens
        l = 1000 
        b = 10
        h = 30          
        
        # Überprüfen und sortieren der Trennstellen
        divisions = set(divisions)
        divisions = list(np.sort(list(divisions)))
        divisions.insert(0, 0)
        divisions.append(l)
        intervals = []
        for x in range(len(divisions)-1):
            intervals.append([divisions[x], divisions[x+1]])
        
        I_f_t, I_m_t, I_f_b, I_m_b, I_f_w, I_m_w = ([] for i in range(6))
        for i in intervals:
            self.mapdl.esel('s','cent','y',i[0],i[1])
            self.mapdl.esel('r','cent','z',h/2)
            F, M = fc_puck(self.mapdl)
            I_f_t.append(F)
            I_m_t.append(M)
            
            self.mapdl.esel('s','cent','y',i[0],i[1])
            self.mapdl.esel('r','cent','z',-h/2)
            F, M = fc_puck(self.mapdl)
            I_f_b.append(F)
            I_m_b.append(M)
            
            self.mapdl.esel('s','cent','y',i[0],i[1])
            self.mapdl.esel('r','cent','x',b)
            F, M = fc_puck(self.mapdl)
            I_f_w.append(F)
            I_m_w.append(M)
        I = list(I_f_t + I_f_b + I_f_w + I_m_t + I_m_b + I_m_w)
        
        self.mapdl.allsel('all')
        self.mapdl.get('M','elem','0','mtot','z')
        mtot = self.mapdl.parameters['M']
        
        return mtot, I
    
    def evaluate(self, x):
    
        x = np.array(x)
        divisions = list(x[0:2]*5)
        if divisions[0] == divisions[1]:
            divisions[0] = divisions[0]-5
        topspar = list(x[2:5])
        bottomspar = list(x[5:8])
        web = list(x[8:11])
        
        self.cdread()
        self.change_design_variables(divisions, topspar, bottomspar, web)
        self.solve()
        mtot, I = self.post_processing(divisions)
        self.clear()
        
        return mtot, I
        
    @property
    def materials(self):
        """
        The Materials assigned to the Model.

        Returns
        -------
        materials : Dict

        """
        return self.__materials
    
    @materials.setter
    def materials(self, materials):
        self.__materials = materials
        
        for key in materials:
            self.__materials[key].load_from_db()
            
    @property
    def mesh_density_factor(self):
        """
        Integer factor to control the mesh density.

        Returns
        -------
        mesh_density_factor : int

        """
        return self.__mesh_density_factor
            
    @mesh_density_factor.setter
    def mesh_density_factor(self, mesh_density_factor):
        self.__mesh_density_factor = mesh_density_factor

In [3]:
ansys_path = '/home/y0065120/Dokumente/Leichtwerk/Projects/ansys-1/'
jobname ='job-0'

mapdl = pyansys.launch_mapdl(run_location=ansys_path,
                            nproc=1,
                            override=True,
                            loglevel='error',
                            additional_switches='-smp -d X11C',
                            jobname=jobname,
                            allow_ignore=True,
                            mode='console',
                            )

femodel = Femodel(mapdl)

femodel.materials = {'flaxpreg': Material(mapdl, 'FLAXPREG-T-UD', 1)}

femodel.pre_processing()

In [4]:
def objfunc(x):
    mtot, I = femodel.evaluate(x)
    
    # Get objective and constraint vector
    f = np.round(mtot*10**6,3)
    g = list(np.round(np.array(I)-1,3))
    
    # Print current Function Evaluation for monitoring purpuses
    objfunc.counter+= 1
    print(objfunc.counter,str(np.round(f,2)).zfill(5),str(int(x[0])).zfill(3),str(int(x[1])).zfill(3),list(np.round(x[2:],3)))
    
    time.sleep(0.01)
    fail = 0
    return f, g, fail
objfunc.counter = 0

In [5]:
opt_prob = Optimization('Faserverbundbalken',objfunc)

# Add variables
t_layer = 0.185
opt_prob.addVar('y1', 'i', lower=4, upper=100, value=30)
opt_prob.addVar('y2', 'i', lower=4, upper=150, value=90)
opt_prob.addVar('n_top1' ,'c', lower=1*0.1, upper=1.5, value=3*t_layer)
opt_prob.addVar('n_top2' ,'c', lower=1*0.1, upper=1.5, value=3*t_layer)
opt_prob.addVar('n_top3' ,'c', lower=1*t_layer, upper=0.75, value=2*t_layer)
opt_prob.addVar('n_bot1' ,'c', lower=1*0.1, upper=1.5, value=5*t_layer)
opt_prob.addVar('n_bot2' ,'c', lower=1*0.1, upper=1.5, value=4*t_layer)
opt_prob.addVar('n_bot3' ,'c', lower=1*t_layer, upper=0.75, value=2*t_layer)
opt_prob.addVar('n_web1' ,'c', lower=1*t_layer, upper=1.5, value=2*t_layer)
opt_prob.addVar('n_web2' ,'c', lower=1*t_layer, upper=1.5, value=1*t_layer)
opt_prob.addVar('n_web3' ,'c', lower=1*t_layer, upper=0.75, value=1*t_layer)

# Add objective
opt_prob.addObj('f')

# Add constraints
opt_prob.addConGroup('g_fib_top', 3, 'i')
opt_prob.addConGroup('g_fib_bot', 3, 'i')
opt_prob.addConGroup('g_fib_web', 3, 'i')
opt_prob.addConGroup('g_mat_top', 3, 'i')
opt_prob.addConGroup('g_mat_bot', 3, 'i')
opt_prob.addConGroup('g_mat_web', 3, 'i')

In [6]:
# Instantiate Optimizer
alpso = ALPSO()
alpso.setOption('fileout',1)

alpso_path = "/home/y0065120/Dokumente/Leichtwerk/Projects/ALPSO/"
filename = 'Balken_Output_ALPSO'

alpso.setOption('filename', alpso_path+filename)
alpso.setOption('SwarmSize', 40)
alpso.setOption('stopIters', 2)      
alpso.setOption('rinit', 1.)
alpso.setOption('itol', 0.01)

def coldstart():    
    alpso(opt_prob, store_hst=True)
    print(opt_prob.solution(0))

In [7]:
coldstart()

1 40.45 036 087 [0.395, 1.445, 0.215, 0.93, 0.625, 0.254, 1.28, 0.898, 0.373]
2 37.83 090 040 [0.604, 0.829, 0.603, 1.386, 0.254, 0.495, 0.761, 0.905, 0.265]
3 46.49 005 133 [0.504, 1.304, 0.564, 0.727, 1.111, 0.222, 1.364, 0.708, 0.193]
4 036.9 035 072 [0.318, 0.133, 0.326, 1.27, 1.04, 0.35, 0.486, 0.606, 0.654]
5 32.98 044 007 [1.424, 1.032, 0.26, 0.947, 0.809, 0.622, 0.878, 1.109, 0.282]
6 38.69 038 037 [0.966, 0.33, 0.547, 0.705, 0.854, 0.598, 0.206, 0.944, 0.633]
7 45.65 072 046 [1.121, 0.719, 0.475, 0.144, 0.705, 0.654, 1.165, 0.744, 0.594]
8 43.97 085 136 [0.716, 0.226, 0.313, 0.555, 1.202, 0.448, 1.071, 0.656, 0.303]
9 42.82 077 088 [0.257, 0.743, 0.45, 0.334, 1.273, 0.456, 0.907, 0.716, 0.719]
10 037.5 053 091 [1.196, 1.421, 0.626, 0.225, 0.368, 0.435, 0.715, 1.017, 0.231]
11 34.56 050 113 [1.042, 0.963, 0.197, 0.866, 0.913, 0.363, 0.364, 0.625, 0.324]
12 37.99 036 064 [0.473, 1.416, 0.324, 0.499, 1.011, 0.309, 0.752, 1.141, 0.503]
13 38.42 084 106 [0.253, 0.964, 0.223, 1.479,

KeyboardInterrupt: 